In [1]:
import sys

!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow

In [2]:
TABULARTRAINING = 'KORL_avatar_train.csv'
TABULARTEST = 'KORL_avatar_test_X.csv'
IMAGELOCATION = r'.'

In [3]:
import numpy as np
import matplotlib.pylab as plt
import cv2
import pandas as pd

%matplotlib inline

In [4]:
# Load the tabular data for references
# needed for appending

train_df = pd.read_csv(TABULARTRAINING)

In [5]:
train_df.head()

,Patient_ID,OMS,Sexe,Age,Date_biopsie,Age_diag,Date_derniere_nouvelles,Deces,Recidive,localisation,RNA_scope,T,N,M,Tabac,Alcool,OS
0,824635,0,1,1965-05-11,2014-03-28,48,2019-11-06,0,0,Oropharynx,2,1,2a,0,3,1,61
1,829316,1,0,1942-12-06,2014-01-01,71,2018-01-09,1,0,Oropharynx,2,3,2b,0,0,1,56
2,82dfbb,1,1,1963-02-05,2012-03-28,48,2020-08-27,1,0,Oropharynx,1,4,2,0,0,1,101
3,832c5f,1,1,1957-11-27,2013-10-25,55,2014-10-25,1,1,Oropharynx,2,2,2b,0,2,1,12
4,837956,0,1,1959-03-15,2012-06-09,53,2021-04-06,0,0,Oropharynx,1,4,1,0,3,1,103


In [6]:
# Convert the images into a numpy data
    
from PIL import Image
from numpy import asarray
import os

path = IMAGELOCATION
list_of_files = []


for root, dirs, files in os.walk(path):
    for file in files:
        list_of_files.append(os.path.join(root,file))
        
l = []
m = None
n = 0
Y = []
YDead = []
for name in list_of_files:
    # load the image and convert into 
    # numpy array
    try:
        img = Image.open(name)
        numpydata = asarray(img)
        
        
        
        # Connect the meta data here
        Y.append(((train_df.loc[train_df['Patient_ID'] == (name.split('_')[0].split('/')[2])])['OS']).item())
        YDead.append(((train_df.loc[train_df['Patient_ID'] == (name.split('_')[0].split('/')[2])])['Deces']).item())

        l.append(numpydata)
    except Exception as e:
        pass


arr = np.array(l)

print(Y)

#DESIRED: [(x, 1404, 1872, 3]

[78, 103, 25, 28, 67, 62, 81, 66, 50, 26, 29, 67, 8, 101, 4, 50, 62, 71, 13, 5, 75, 48, 80, 29, 77, 48, 7, 4, 8, 81, 78, 4, 78, 19, 21, 75, 19, 1, 81, 7, 46, 8, 12, 65, 80, 26, 68, 26, 82, 82, 27, 19, 67, 28, 25, 27, 80, 71, 62, 46, 78, 101, 65, 4, 81, 66, 19, 25, 56, 19, 26, 71, 15, 103, 21, 1, 80, 66, 75, 19, 75, 67, 46, 62, 8, 101, 80, 77, 4, 28, 82, 82, 78, 50, 29, 65, 68, 81, 67, 103, 19, 27, 80, 67, 19, 66, 25, 101, 77, 19, 12, 28, 78, 12, 19, 81, 1, 80, 103, 48, 13, 67, 78, 1, 80, 81, 81, 21, 67, 12, 77, 29, 46, 27, 65, 50, 48]


In [7]:
Y = np.asarray(Y)
YDead = np.asarray(YDead)

In [8]:
print(Y)

[ 78 103  25  28  67  62  81  66  50  26  29  67   8 101   4  50  62  71
  13   5  75  48  80  29  77  48   7   4   8  81  78   4  78  19  21  75
  19   1  81   7  46   8  12  65  80  26  68  26  82  82  27  19  67  28
  25  27  80  71  62  46  78 101  65   4  81  66  19  25  56  19  26  71
  15 103  21   1  80  66  75  19  75  67  46  62   8 101  80  77   4  28
  82  82  78  50  29  65  68  81  67 103  19  27  80  67  19  66  25 101
  77  19  12  28  78  12  19  81   1  80 103  48  13  67  78   1  80  81
  81  21  67  12  77  29  46  27  65  50  48]


In [9]:
X = arr # images

In [10]:
print(X.shape)
print(Y.shape)

(137, 1404, 1872, 3)
(137,)


In [11]:
# How many images ?

print('Total number of images : {}'.format(len(X)))
print('Total number of LABELS : {}'.format(len(Y)))

# What is the size of the images ?

print('Image shape (number of rows, number of columns, channels RGB): {}'.format(X[0].shape))

Total number of images : 137
Total number of LABELS : 137
Image shape (number of rows, number of columns, channels RGB): (1404, 1872, 3)


In [12]:
from sklearn.model_selection import StratifiedKFold as KFold

# Prepare the folds for a cross validation. We use the very useful scikit learn package. Here we use stratified 
# cross validation : the proportion of positive and negative examples is the same in all folds

N = len(X)
folds = lambda : KFold(n_splits = 5, shuffle = True, random_state=0).split(X, Y)

# Let's have a look to the split size and 

for i, (train_indexes, test_indexes) in enumerate(folds()):
    print('split {}: training : {} images, test : {} images'.format(i + 1, len(train_indexes), len(test_indexes)))

split 1: training : 109 images, test : 28 images
split 2: training : 109 images, test : 28 images
split 3: training : 110 images, test : 27 images
split 4: training : 110 images, test : 27 images
split 5: training : 110 images, test : 27 images


/home/vikram/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(109, 1404, 1872, 3)
(28, 1404, 1872, 3)
(109,)
(28,)


# BUILD FEATURES WITH RESNET50

In [14]:
from keras.applications.resnet import ResNet50, preprocess_input
import cv2

model = ResNet50(weights='imagenet', include_top=False, pooling='max')
X_224 = np.array([cv2.resize(xi, (224, 224)) for xi in X])
X_224 = preprocess_input(X_224.astype('float'))
resnet_features = model.predict(X_224, verbose=1)

5/5 [==============================] - 24s 4s/step


In [15]:
resnet_features.shape

(137, 2048)

# BUILD WITH XGBOOST

In [16]:
# append meta data to the resnet features



In [17]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(resnet_features, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

RegModel=XGBRegressor(max_depth=3, learning_rate=0.5, n_estimators=500, objective='reg:linear', booster='gbtree')

#Printing all the parameters of XGBoost
print(RegModel)

#Creating the model on Training Data
XGB=RegModel.fit(X_train,y_train)
prediction=XGB.predict(X_test)
prediction = np.absolute(prediction)

#Measuring Goodness of fit in Training data
from sklearn import metrics
print('MAPE Value:',metrics.mean_absolute_percentage_error(y_test, prediction))

#Measuring accuracy on Testing Data
print(prediction)
print(y_train)

(109, 2048)
(28, 2048)
(109,)
(28,)
XGBRegressor(base_score=None, booster='gbtree', colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None,
             enable_categorical=False, gamma=None, gpu_id=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.5, max_delta_step=None, max_depth=3,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', predictor=None, random_state=None,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
[19:27:19] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
MAPE Value: 4.933228718524425
[64.20965  51.808483 44.050663 37.156628 69.40395  52.22718  73.51149
 49.60053  65.10346  48.32347  2

In [18]:
!mkdir -p saved_model
XGB.save_model("layer3.json")